In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt



In [6]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [7]:
# Set random seeds for reproducibility
torch.manual_seed(42)


In [9]:
df = pd.read_csv('fmnist_small.csv')
df.head(5)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [10]:
df.shape

(6000, 785)

In [11]:
#train test
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [12]:
X

array([[  0,   0,   0, ..., 165,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]])

In [13]:
y

array([9, 7, 0, ..., 8, 4, 8])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
#scalling
X_train = X_train/255.0
X_test = X_test/255.0


In [16]:
#create custom dataset
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels =  torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]

In [17]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# **Hyperparameters Tunning**

In [29]:
# define NN class
class MyNN(nn.Module):

  def __init__(self, input_size, output_size, num_hidden_layers, neurons_per_layers, dropout_rate):
    super().__init__()

    layers = []

    for i in range(num_hidden_layers):
      layers.append(nn.Linear(input_size, neurons_per_layers))
      layers.append(nn.BatchNorm1d(neurons_per_layers))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))
      input_size = neurons_per_layers

    layers.append(nn.Linear(input_size, output_size))

    self.model = nn.Sequential(*layers)


  def forward(self, x):

    return self.model(x)

In [26]:
def objective(trial):
  # next hyperparameter values from the search space
  num_hidden_layers = trial.suggest_int("num_hidden_layers", 1, 5)
  neurons_per_layers = trial.suggest_int("neurons_per_layer", 8, 128, step=8)
  epochs = trial.suggest_int("epochs", 10, 50, step=10)
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  dropout_rate = trial.suggest_float('dropout_rate',0.1, 0.5, step=0.1)
  batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])
  optimizer_name = trial.suggest_categorical("optimizer", ['Adam', 'SGD', 'RMSprop'])
  weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

  # model init
  input_dim = 784
  output_dim = 10

  model = MyNN(input_dim, output_dim, num_hidden_layers, neurons_per_layers, dropout_rate)
  model.to(device)

  #optimizer selection
  criterion = nn.CrossEntropyLoss()

  if optimizer_name == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  elif optimizer_name == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  else:
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  for epoch in range(epochs):

    for batch_features, batch_labels in train_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      # forward pass
      outputs = model(batch_features)

      # calculate loss
      loss = criterion(outputs, batch_labels)

      # back pass
      optimizer.zero_grad()
      loss.backward()

      # update grads
      optimizer.step()


  # evaluation
  model.eval()
  # evaluation on test data
  total = 0
  correct = 0

  with torch.no_grad():

    for batch_features, batch_labels in test_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      outputs = model(batch_features)

      _, predicted = torch.max(outputs, 1)

      total = total + batch_labels.shape[0]

      correct = correct + (predicted == batch_labels).sum().item()

    accuracy = correct/total

  return accuracy




In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 20.5 MB/s eta 0:00:00


In [27]:
import optuna
study = optuna.create_study(direction='maximize')


[I 2025-06-08 07:11:19,152] A new study created in memory with name: no-name-ee20f6c3-a5bc-416d-b3a5-eae020f71b58


In [30]:
study.optimize(objective, n_trials=10)

[I 2025-06-08 07:12:23,733] Trial 1 finished with value: 0.8325 and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 48, 'epochs': 40, 'learning_rate': 0.04588072542923886, 'dropout_rate': 0.2, 'batch_size': 16, 'optimizer': 'SGD', 'weight_decay': 7.474786393174004e-05}. Best is trial 1 with value: 0.8325.
[I 2025-06-08 07:12:26,475] Trial 2 finished with value: 0.815 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 48, 'epochs': 10, 'learning_rate': 8.593680679784006e-05, 'dropout_rate': 0.30000000000000004, 'batch_size': 32, 'optimizer': 'RMSprop', 'weight_decay': 2.375990944706231e-05}. Best is trial 1 with value: 0.8325.
[I 2025-06-08 07:12:28,812] Trial 3 finished with value: 0.7875 and parameters: {'num_hidden_layers': 2, 'neurons_per_layer': 48, 'epochs': 20, 'learning_rate': 0.0010821464560586023, 'dropout_rate': 0.5, 'batch_size': 128, 'optimizer': 'Adam', 'weight_decay': 1.2509142842832245e-05}. Best is trial 1 with value: 0.8325.
[I 2025-06-08 07:12:32,

In [32]:
study.best_value

0.8325

In [31]:
study.best_params

{'num_hidden_layers': 4,
 'neurons_per_layer': 48,
 'epochs': 40,
 'learning_rate': 0.04588072542923886,
 'dropout_rate': 0.2,
 'batch_size': 16,
 'optimizer': 'SGD',
 'weight_decay': 7.474786393174004e-05}